# Get players from tennisabstract.com

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
PLAYER_URL = "https://tennisabstract.com/reports/atpRankings.html"
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [3]:
response = requests.get(PLAYER_URL, headers=headers)
response.status_code

In [5]:
players = pd.DataFrame(columns=["player_id", "rank","name", "nationality", "date_of_birth"])
player_data = [] 

In [5]:
if response.status_code == 200:
    x = 0
    soup = BeautifulSoup(response.content, "html.parser")
    player_table = soup.find("table", class_="tablesorter")
    if player_table:
        tr_player_elements = player_table.find_all("tr")
        first_row_skipped = False
        
        for player_elem in tr_player_elements:
            if not first_row_skipped:
                first_row_skipped = True
                continue
            td_elems = player_elem.find_all("td")
            if td_elems:
                rank = td_elems[0].text.strip() 
                player_url = td_elems[1].find("a")["href"] 
                player_name = td_elems[1].text.strip().replace("\xa0", " ")  
                nationality = td_elems[2].text.strip()
                dob = td_elems[3].text.strip() 
                player_data.append({
                    "player_id": x,
                    "rank": rank,
                    "player_url": player_url,
                    "name": player_name,
                    "nationality": nationality,
                    "date_of_birth": dob
                })
                x+=1
players = pd.concat([players, pd.DataFrame(player_data)], ignore_index=True)

# Display the DataFrame
players

In [15]:
players

In [16]:
players.to_csv("../data/player_urls.csv")